# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Dec 30 2022 8:30AM,254027,10,MSP217113,"Methapharm, Inc.",Released
1,Dec 30 2022 8:30AM,254025,10,8965028,Emerginnova,Released
2,Dec 30 2022 8:30AM,254026,10,SNAP33833535,"Snap Medical Industries, LLC",Released
3,Dec 29 2022 2:29PM,254022,10,MSP217106,"Methapharm, Inc.",Released
4,Dec 29 2022 2:29PM,254022,10,MSP217108,"Methapharm, Inc.",Released
5,Dec 29 2022 2:29PM,254022,10,MSP217110,"Methapharm, Inc.",Released
6,Dec 29 2022 2:29PM,254022,10,MSP217111,"Methapharm, Inc.",Released
7,Dec 29 2022 2:21PM,254021,10,0086265534,ISDIN Corporation,Executing
8,Dec 29 2022 1:47PM,254015,10,0086271569,ISDIN Corporation,Released
9,Dec 29 2022 1:47PM,254015,10,0086271571,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
24,254021,Executing,1
25,254022,Released,4
26,254025,Released,1
27,254026,Released,1
28,254027,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
254021,1.0,NaN
254022,NaN,4.0
254025,NaN,1.0
254026,NaN,1.0
254027,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
253953,0.0,1.0
253960,0.0,1.0
253964,0.0,4.0
253965,0.0,14.0
253967,0.0,6.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
253953,0,1
253960,0,1
253964,0,4
253965,0,14
253967,0,6


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,253953,0,1
1,253960,0,1
2,253964,0,4
3,253965,0,14
4,253967,0,6


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,253953,,1
1,253960,,1
2,253964,,4
3,253965,,14
4,253967,,6


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Dec 30 2022 8:30AM,254027,10,"Methapharm, Inc."
1,Dec 30 2022 8:30AM,254025,10,Emerginnova
2,Dec 30 2022 8:30AM,254026,10,"Snap Medical Industries, LLC"
3,Dec 29 2022 2:29PM,254022,10,"Methapharm, Inc."
7,Dec 29 2022 2:21PM,254021,10,ISDIN Corporation
8,Dec 29 2022 1:47PM,254015,10,ISDIN Corporation
57,Dec 29 2022 1:46PM,254017,10,"Methapharm, Inc."
67,Dec 29 2022 1:46PM,254019,10,Emerginnova
68,Dec 29 2022 1:45PM,254018,10,Yusen – 3D Matrix
69,Dec 29 2022 1:45PM,254016,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Dec 30 2022 8:30AM,254027,10,"Methapharm, Inc.",,1
1,Dec 30 2022 8:30AM,254025,10,Emerginnova,,1
2,Dec 30 2022 8:30AM,254026,10,"Snap Medical Industries, LLC",,1
3,Dec 29 2022 2:29PM,254022,10,"Methapharm, Inc.",,4
4,Dec 29 2022 2:21PM,254021,10,ISDIN Corporation,1,
5,Dec 29 2022 1:47PM,254015,10,ISDIN Corporation,,49
6,Dec 29 2022 1:46PM,254017,10,"Methapharm, Inc.",,10
7,Dec 29 2022 1:46PM,254019,10,Emerginnova,,1
8,Dec 29 2022 1:45PM,254018,10,Yusen – 3D Matrix,,1
9,Dec 29 2022 1:45PM,254016,10,ISDIN Corporation,,3


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Dec 30 2022 8:30AM,254027,10,"Methapharm, Inc.",1,
1,Dec 30 2022 8:30AM,254025,10,Emerginnova,1,
2,Dec 30 2022 8:30AM,254026,10,"Snap Medical Industries, LLC",1,
3,Dec 29 2022 2:29PM,254022,10,"Methapharm, Inc.",4,
4,Dec 29 2022 2:21PM,254021,10,ISDIN Corporation,,1
5,Dec 29 2022 1:47PM,254015,10,ISDIN Corporation,49,
6,Dec 29 2022 1:46PM,254017,10,"Methapharm, Inc.",10,
7,Dec 29 2022 1:46PM,254019,10,Emerginnova,1,
8,Dec 29 2022 1:45PM,254018,10,Yusen – 3D Matrix,1,
9,Dec 29 2022 1:45PM,254016,10,ISDIN Corporation,3,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Dec 30 2022 8:30AM,254027,10,"Methapharm, Inc.",1,
1,Dec 30 2022 8:30AM,254025,10,Emerginnova,1,
2,Dec 30 2022 8:30AM,254026,10,"Snap Medical Industries, LLC",1,
3,Dec 29 2022 2:29PM,254022,10,"Methapharm, Inc.",4,
4,Dec 29 2022 2:21PM,254021,10,ISDIN Corporation,,1


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Dec 30 2022 8:30AM,254027,10,"Methapharm, Inc.",1.0,NaN
1,Dec 30 2022 8:30AM,254025,10,Emerginnova,1.0,NaN
2,Dec 30 2022 8:30AM,254026,10,"Snap Medical Industries, LLC",1.0,NaN
3,Dec 29 2022 2:29PM,254022,10,"Methapharm, Inc.",4.0,NaN
4,Dec 29 2022 2:21PM,254021,10,ISDIN Corporation,NaN,1.0


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Dec 30 2022 8:30AM,254027,10,"Methapharm, Inc.",1.0,0.0
1,Dec 30 2022 8:30AM,254025,10,Emerginnova,1.0,0.0
2,Dec 30 2022 8:30AM,254026,10,"Snap Medical Industries, LLC",1.0,0.0
3,Dec 29 2022 2:29PM,254022,10,"Methapharm, Inc.",4.0,0.0
4,Dec 29 2022 2:21PM,254021,10,ISDIN Corporation,0.0,1.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,6096030,207.0,1.0
19,761961,3.0,0.0
21,508001,1.0,1.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,6096030,207.0,1.0
1,19,761961,3.0,0.0
2,21,508001,1.0,1.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,207.0,1.0
1,19,3.0,0.0
2,21,1.0,1.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,207.0
1,19,Released,3.0
2,21,Released,1.0
3,10,Executing,1.0
4,19,Executing,0.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,19,21
Status,,,
Executing,1.0,0.0,1.0
Released,207.0,3.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,19,21
0,Executing,1.0,0.0,1.0
1,Released,207.0,3.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,19,21
0,Executing,1.0,0.0,1.0
1,Released,207.0,3.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()